In [1]:
!pip install gymnasium mujoco stable-baselines3 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
# Import the necessary libraries
import gymnasium as gym
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt

In [3]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from google.colab import data_table
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


Mounted at /content/drive


In [4]:
# Load the dataframe back from the pickle file
trajectory_df = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/02_trajectories/0_initial_training/0_initial_trajectory_reacher_df.pkl')     # Update directory location 1

In [5]:
# Display the data frame
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df)

Output hidden; open in https://colab.research.google.com to view.

In [6]:
# Check the data type of each column
print(type(trajectory_df['Episode'][0]))
print(type(trajectory_df['Timestep'][0]))
print(type(trajectory_df['State'][0]))
print(type(trajectory_df['Action'][0]))
print(type(trajectory_df['Reward'][0]))
print(type(trajectory_df['Next State'][0]))

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float64'>
<class 'numpy.ndarray'>


In [7]:
trajectory_df.dtypes

,0
Episode,int64
Timestep,int64
State,object
Action,object
Reward,float64
Next State,object


HUMAN FEEDBACK IMPLEMENTATION (IDEAL CASE SCENARIO)



In [8]:
def calculate_different_rewards(trajectory_df):
    """Calculates effort-based reward and returns updated dataframe."""
    df = trajectory_df.copy()

    df['Distance Reward'] = -df['State'].apply(lambda x: np.linalg.norm(np.array(x)[8:10], axis=0))

    df['Smoothness Reward'] = -np.abs(df['State'].apply(lambda x: np.array(x)[6:8]).apply(np.linalg.norm))

    # df['Effort Reward'] = -df['Action'].apply(lambda x: np.linalg.norm(np.array(x)**2, axis=0))
    df['Effort Reward'] = -df['Action'].apply(lambda x: np.linalg.norm(np.array(x), axis=0))
    return df

In [9]:
different_rewards_df = calculate_different_rewards(trajectory_df)

In [10]:
data_table.enable_dataframe_formatter()
data_table.DataTable(different_rewards_df)

Output hidden; open in https://colab.research.google.com to view.

In [11]:
def compute_weighted_rewards(df):
    # Step 1: Create a copy of the dataframe
    df = df.copy()

    # Step 2: Compute min and max values for Smoothness Reward and Effort Reward
    min_smoothness = df["Smoothness Reward"].min()
    max_smoothness = df["Smoothness Reward"].max()
    print("Minimum Smoothness Reward",min_smoothness)
    print("Maximum Smoothness Reward",max_smoothness)

    min_effort = df["Effort Reward"].min()
    max_effort = df["Effort Reward"].max()
    print("Minimum Effort Reward",min_effort)
    print("Maximum Effort Reward",max_effort)

    # Step 3: Compute Smoothness Score Range and Effort Score Range
    xsmooth = 1 / abs(max_smoothness - min_smoothness)
    xeffort = 1 / abs(max_effort - min_effort)

    print(f"Smoothness Score Range: {xsmooth}")
    print(f"Effort Score Range: {xeffort}")

    # Step 4: Compute Smoothness Weight (λ) and Effort Weight (ε) using Softmax
    exp_xsmooth = np.exp(xsmooth)
    exp_xeffort = np.exp(xeffort)

    lambda_smooth = exp_xsmooth / (exp_xsmooth + exp_xeffort)                      # bias value 1
    epsilon_effort = exp_xeffort / (exp_xsmooth + exp_xeffort)                     # bias value 2

    print(f"Smoothness Weight (λ): {lambda_smooth}")
    print(f"Effort Weight (ε): {epsilon_effort}")

    # Step 5: Compute Smoothness Weighted Reward (R1) and Effort Weighted Reward (R2)
    df["Smoothness Weighted Reward"] = -df["Distance Reward"] - lambda_smooth * df["Smoothness Reward"]
    df["Effort Weighted Reward"] = -df["Distance Reward"] - epsilon_effort * df["Effort Reward"]

    # df["Smoothness Weighted Reward"] = df["Distance Reward"] + lambda_smooth * df["Smoothness Reward"]
    # df["Effort Weighted Reward"] = df["Distance Reward"] + epsilon_effort * df["Effort Reward"]


    return df

In [12]:
reward_df = compute_weighted_rewards(different_rewards_df)

Minimum Smoothness Reward -9.647165745920095
Maximum Smoothness Reward -0.000848030583177548
Minimum Effort Reward -0.5573621988296509
Maximum Effort Reward -0.0008373789023607969
Smoothness Score Range: 0.10366650047303289
Effort Score Range: 1.7968650529021328
Smoothness Weight (λ): 0.15535566234653136
Effort Weight (ε): 0.8446443376534686


In [13]:
data_table.enable_dataframe_formatter()
data_table.DataTable(reward_df)

Output hidden; open in https://colab.research.google.com to view.

In [26]:
# Function to recalibrate the rewards
def recalibrate_rewards(df):
    # Create a copy of the dataframe
    df_copy = df.copy()
    alpha = 1
    # Create the 'Recalibrated_rewards' column
    df_copy['Recalibrated Reward'] = df_copy['Reward'] + alpha * (df_copy['Smoothness Weighted Reward'] + df_copy['Effort Weighted Reward'])

    # Get the list of recalibrated rewards
    recalibrated_rewards_list = df_copy['Recalibrated Reward'].tolist()

    return df_copy, recalibrated_rewards_list

In [27]:
# Apply the function to recalibrate rewards
recalibrated_df, recalibrated_rewards_list = recalibrate_rewards(reward_df)

In [28]:
data_table.enable_dataframe_formatter()
data_table.DataTable(recalibrated_df)

Output hidden; open in https://colab.research.google.com to view.

In [30]:
recalibrated_df.to_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/02_trajectories/1_human_feedback/1_Hf_D_Ideal_Reacher_df_test_3.pkl')         # Update directory location 2

In [29]:
# To access the reward for a specific step:
for i, feedback in enumerate(recalibrated_rewards_list):
    human_recalibrated_reward_for_step = recalibrated_rewards_list[i]
    print(f"Recalibrated reward for step {i}: {human_recalibrated_reward_for_step}")

Recalibrated reward for step 0: 0.17553261398985623
Recalibrated reward for step 1: 0.23551766009325825
Recalibrated reward for step 2: 0.3196225012925384
Recalibrated reward for step 3: 0.4323377280624078
Recalibrated reward for step 4: 0.4369206055313877
Recalibrated reward for step 5: 0.4211856961771358
Recalibrated reward for step 6: 0.38914151485800125
Recalibrated reward for step 7: 0.41368808190056616
Recalibrated reward for step 8: 0.49178798109890326
Recalibrated reward for step 9: 0.5251248675370938
Recalibrated reward for step 10: 0.6024730921611731
Recalibrated reward for step 11: 0.6415234783481167
Recalibrated reward for step 12: 0.6371690060789478
Recalibrated reward for step 13: 0.6900205135065989
Recalibrated reward for step 14: 0.6938467504764869
Recalibrated reward for step 15: 0.7229435412921775
Recalibrated reward for step 16: 0.664911322672586
Recalibrated reward for step 17: 0.7196571870918821
Recalibrated reward for step 18: 0.7525615569279571
Recalibrated rewar

In [31]:
# Step A.5.1: CUSTOM REWARD FUNCTION
def custom_reward(self, env, state, action, next_state, reward, done):
    # Access and recalculate the reward using human_feedback_data or recalibrate_rewards_human function
    global step_counter
    try:
        step_counter
    except NameError:
        step_counter = 0

    reward = recalibrated_rewards_list[step_counter]
    step_counter += 1
    return reward

# Create a new environment class that wraps your original environment and overrides the default reward function with your custom function
class CustomRewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super(CustomRewardWrapper, self).__init__(env)

    def step(self, action):
        next_state, reward, terminated, truncated, info = self.env.step(action)
        done = terminated or truncated
        reward = custom_reward(self, self.env, self.last_obs, action, next_state, reward, done)
        # custom_reward should be defined and accessible to your class
        self.last_obs = next_state
        return next_state, reward, terminated, truncated, info

    def reset(self, **kwargs):
        global step_counter
        step_counter = 0
        self.last_obs = self.env.reset(**kwargs)[0]  # Assuming Gymnasium env returns (obs, info)
        return self.last_obs, {}  # Assuming Gymnasium env requires (obs, info)
# Create and wrap the environment with your custom reward wrapper
# env_human = CustomRewardWrapper(gym.make('highway-v0'))

PPO training and Training logs

In [32]:
drive_log_dir = "/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/00_training_logs/1_log_dir/1_ppo_reacher_hf_direct_ideal_test_3"              # Update directory location 3

In [33]:
# Train PPO with Custom Rewards
def train_ppo_with_custom_rewards(log_dir=drive_log_dir, total_timesteps=200000):
    os.makedirs(log_dir, exist_ok=True)
    env = CustomRewardWrapper(gym.make("Reacher-v5"))
    env = Monitor(env, log_dir)
    model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_dir)
    model.learn(total_timesteps=total_timesteps)
    model.save('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/01_trained_models/1_ppo_reacher_hf_direct_ideal_test_3')       # Update directory location 4
    return model, log_dir

In [34]:
# Execute Training and Convergence Tracking
model, log_dir = train_ppo_with_custom_rewards(total_timesteps=200000)

Using cpu device
Wrapping the env in a DummyVecEnv.
Logging to /content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/00_training_logs/1_log_dir/1_ppo_reacher_hf_direct_ideal_test_3/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | 35.9     |
| time/              |          |
|    fps             | 854      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 50          |
|    ep_rew_mean          | 35.9        |
| time/                   |             |
|    fps                  | 724         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.005003359 |
|    cl

In [35]:
log_path = os.path.join(drive_log_dir, "monitor.csv")
df = pd.read_csv(log_path, skiprows=1)
# Ensure episodes are logged correctly
df.reset_index(inplace=True)
df.rename(columns={"index": "episode", "r": "reward", "l": "length", "t": "time_step"}, inplace=True)

In [36]:
data_table.enable_dataframe_formatter()
data_table.DataTable(df.head())

,episode,reward,length,time_step
0,0,35.943083,50,7.584540
1,1,35.943083,50,7.638646
2,2,35.943083,50,7.690836
3,3,35.943083,50,7.741987
4,4,35.943083,50,7.793356
